In [3]:
library(SingleCellExperiment)
library(Seurat)
library(tidyverse)
library(SeuratDisk)

Loading required package: SummarizedExperiment

Attaching SeuratObject


Attaching package: ‘Seurat’


The following object is masked from ‘package:SummarizedExperiment’:

    Assays


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::collapse()   masks IRanges::collapse()
✖ dplyr::combine()    masks Biobase::combine(), BiocGenerics::combine()
✖ dplyr::count()      masks matrixStats::count()
✖ dplyr::desc()       masks IRanges::desc()
✖ tidyr::expand()     masks S4Vectors::expand()
✖ dplyr::filter()     masks stats::filter()
✖ dplyr::first()      masks S4Vectors::first()
✖ dplyr::lag()        masks stats::lag()
✖ ggplot2::Position() masks BiocGenerics::Position(), base::Position()
✖ purrr::reduce()     masks GenomicRa

In [4]:
source('../00_preprocessing/preprocessing.R')
source('../01_milo/milo.R')

In [5]:
# obj_neurons = readRDS('/projects/mludwig/DVC/output/Seurat_objs/integrated/neurons_Seurat_obj_labels.rds')
# obj_neurons
# obj_neurons %>% qs::qsave('dvc_neurons.qs')

In [24]:
obj_neurons = qs::qread('dvc_neurons.qs')
DefaultAssay(obj_neurons) = 'rat_RNA'
obj_neurons[['SCT']] = NULL
obj_neurons[['RNA']] = NULL
obj_neurons[['integrated']] = NULL
obj_neurons[['mouse_RNA']] = NULL
obj_neurons[['macaque_RNA']] = NULL
obj_neurons[['pca']] = NULL
obj_neurons[['umap']] = NULL
obj_neurons

An object of class Seurat 
32883 features across 218547 samples within 1 assay 
Active assay: rat_RNA (32883 features, 0 variable features)

In [25]:
obj_neurons %>%
`[[` %>%
filter(species == 'rat') %>%
group_by(run) %>%
summarise(n = n())

run,n
<chr>,<int>
rat1,17587
rat2,21171
rat3,20713
rat4,21721
rat5,16370
rat6,24306


In [26]:
rat_cells = obj_neurons %>%
`[[` %>%
filter(species == 'rat') %>%
rownames

In [27]:
length(rat_cells)

[1] 121868

In [28]:
obj = obj_neurons %>%
      subset(cells = rat_cells) 
obj

An object of class Seurat 
32883 features across 121868 samples within 1 assay 
Active assay: rat_RNA (32883 features, 0 variable features)

In [29]:
obj = obj %>%
      Seurat::SCTransform(assay='rat_RNA',
                          method="glmGamPoi",
                          verbose=TRUE,
                          variable.features.n = 10000) %>%
      RunPCA(npcs = 100) %>%
      RunUMAP(dims = 1:30)
obj

Calculating cell attributes from input UMI matrix: log_umi

Variance stabilizing transformation of count matrix of size 25537 by 121868

Model formula is y ~ log_umi

Get Negative Binomial regression parameters per gene

Using 2000 genes, 5000 cells



  |======================================================================| 100%


Found 89 outliers - those will be ignored in fitting/regularization step


Second step: Get residuals using fitted parameters for 25537 genes



  |======================================================================| 100%


Computing corrected count matrix for 25537 genes



  |======================================================================| 100%


Calculating gene attributes

Wall clock passed: Time difference of 41.0571 mins

Determine variable features

Place corrected count matrix in counts slot

Centering data matrix

Set default assay to SCT

Warning message:
“Invalid name supplied, making object name syntactically valid. New object name is Seurat..SCTransform.rat_RNA; see ?make.names for more details on syntax validity”
PC_ 1 
Positive:  Lrrtm4, Il1rapl2, Il1rapl1, Htr2c, Hs6st3, LOC304725, Grm8, AABR07060487.1, AABR07057765.1, Rbfox1 
	   AC134224.1, Gpc6, Cacna2d3, Brinp3, Gria4, Adgrb3, Cadps, Kcnip1, Lrfn5, Dgkb 
	   Mctp1, RGD1564053, Cdh12, Gria3, Cntnap5c, Ano4, Ndst4, Atp8a2, Vwc2l, AABR07033882.1 
Negative:  Ubb, Ndufa4, Tmsb4x, Hsp90ab1, Stmn1, Resp18, Atp6v0c, Sst, Caly, Rpl19 
	   Pebp1, Calm1, Stmn3, Slc25a4, LOC100364435, Cox4i1, Aldoa, Fth1, LOC687780, Cox5b 
	   Atp5if1, Tpt1, AABR07000398.1, Rpl23, Penk, Cox7b, Rps8, Tuba1a, Cox6c, Cox6a1 
PC_ 2 
Positive:  Il1rapl2, AABR07033570.1, LOC304725, Nefm, Grik1,

An object of class Seurat 
58420 features across 121868 samples within 2 assays 
Active assay: SCT (25537 features, 10000 variable features)
 1 other assay present: rat_RNA
 2 dimensional reductions calculated: pca, umap

In [30]:
obj %>% qs::qsave('dvc_neurons_rat.qs')

In [31]:
obj

An object of class Seurat 
58420 features across 121868 samples within 2 assays 
Active assay: SCT (25537 features, 10000 variable features)
 1 other assay present: rat_RNA
 2 dimensional reductions calculated: pca, umap

In [32]:
SaveH5Seurat(obj, filename = "/scratch/nmq407/dvc_neurons.h5Seurat", overwrite = TRUE)

Warning message:
“Overwriting previous file /scratch/nmq407/dvc_neurons.h5Seurat”
Creating h5Seurat file for version 3.1.5.9900

Adding counts for rat_RNA

Adding data for rat_RNA

No variable features found for rat_RNA

No feature-level metadata found for rat_RNA

Adding counts for SCT

Adding data for SCT

Adding scale.data for SCT

Adding variable features for SCT

No feature-level metadata found for SCT

Writing out SCTModel.list for SCT

Adding cell embeddings for pca

Adding loadings for pca

No projected loadings for pca

Adding standard deviations for pca

No JackStraw data for pca

Adding cell embeddings for umap

No loadings for umap

No projected loadings for umap

No standard deviations for umap

No JackStraw data for umap



In [33]:
Convert("/scratch/nmq407/dvc_neurons.h5Seurat", dest = "/scratch/nmq407/dvc_neurons.h5Seurat.h5ad")

Validating h5Seurat file

Adding scale.data from SCT as X

Adding data from SCT as raw

Transfering meta.data to obs

Adding dimensional reduction information for pca

Adding feature loadings for pca

Adding dimensional reduction information for umap



In [34]:
2

[1] 2